
# 分类问题和决策树


## 概述

机器学习提供了许多将数据分类为离散类别的方法，例如k-means聚类。决策树为这种分类提供了一种结构，它基于一系列决策导致不同结果。我们将建立和训练能够解决分类问题的决策树，使用决策树根据某些决策边界执行分类。我们首先学习如何构建决策树，然后学习如何有效地训练它们，最后学习如何测试它们的性能。

<p>
<img src="./files/dt.png" alt="Decision Trees" width="700" height="350"/>


* 所有的函数都将在 **_submission.py_** 中完成。

### 重要文件

1. **_submission.py_**: 您将在其中构建决策树、混淆矩阵、性能指标、随机森林，并实现矢量化操作。
2. **_decision_trees_submission_tests.py_**: 用于验证树、学习和矢量化的样例测试。
3. **_unit_testing.ipynb_**: 依次运行测试的辅助 notebook。

### 决策树数据集
     
1. **_hand_binary.csv_**: 4 features, 8 examples, binary classification (last column)
2. **_hand_multi.csv_**: 4 features, 12 examples, 3 classes, multi-class classification (last column)
3. **_simple_binary.csv_**: 5 features, 100 examples, binary classification (last column)
4. **_simple_multi.csv_**: 6 features, 100 examples, 3 classes, multi-class classification (last column)
5. **_mod_complex_binary.csv_**: 7 features, 1400 examples, binary classification (last column)
6. **_mod_complex_multi.csv_**: 10 features, 1800 examples, 5 classes, multi-class classification (last column)
7. **_complex_binary.csv_**: 10 features, 5400 examples, binary classification (last column)
8. **_complex_multi.csv_**: 16 features, 10800 examples, 9 classes, multi-class classification (last column)
   
#### 数据集的文件路径 './data/your_file_name.csv'

#### 矢量化验证数据
    
 **_vectorize.csv_**: 用于对矢量化操作进行验证的数据

### 注意

我们的重点在于实现决策树和随机森林的相关算法，因此只允许导入四个库: numpy, math, collections.Counter, time。


In [5]:
### Setting Up some utilities for testing:
from __future__ import division

import unittest
import submission as dt
import numpy as np
import importlib
import decision_trees_submission_tests

In [6]:
def single_tester(test, case):
    importlib.reload(dt)
    importlib.reload(decision_trees_submission_tests)
    if test == decision_trees_submission_tests.DecisionTreePart1Tests:
        print("Running Decision Tree Part 1 Test: {}".format(case))
    elif test == decision_trees_submission_tests.DecisionTreePart2Tests:
        print("Running Decision Tree Part 2 Test: {}".format(case))
    elif test == decision_trees_submission_tests.DecisionTreePart3Tests:
        print("Running Decision Tree Part 3 Test: {}".format(case))
    elif test == decision_trees_submission_tests.DecisionTreePart4Tests:
        print("Running Decision Tree Part 4 Test: {}".format(case))
    elif test == decision_trees_submission_tests.VectorizationWarmUpTests:
        print("Running Vectoriization Warmup Tests: {}".format(case))
        
    suite = unittest.TestSuite()
    suite.addTest(test(case))
    runner = unittest.TextTestRunner()
    runner.run(suite)

### Part 0: 矢量化

本部分将说明如何使用矩阵来显著提高人工智能问题的处理速度和降低处理复杂性。

机器学习都需要大量使用矢量化操作。NumPy 开源项目提供 NumPy python 科学计算包（使用 C 编写）。

如果没有矢量化，你将会耗费较长的时间，这是非常不明智的。

测试:
我们将未使用矢量化操作的代码和使用矢量化操作的代码运行500次，只要使用矢量化操作的代码的平均时间小于未使用矢量化操作的代码的平均时间，就可以认为矢量化操作是成功的。

#### 在  `Vectorization`  类中需要完成的方法:
1. `vectorized_loops()`
2. `vectorized_slice()`
3. `vectorized_flatten()`
4. `vectorized_glue()`
5. `vectorized_mask()`


1. `vectorized_loops()`

In [7]:
single_tester(decision_trees_submission_tests.VectorizationWarmUpTests, 'test_vectorized_loops')

.
----------------------------------------------------------------------
Ran 1 test in 0.051s

OK


In [8]:
single_tester(decision_trees_submission_tests.VectorizationWarmUpTests, 'test_vectorized_loops_time')

.
----------------------------------------------------------------------
Ran 1 test in 0.014s

OK


2. `vectorized_slice()`

In [9]:
single_tester(decision_trees_submission_tests.VectorizationWarmUpTests, 'test_vectorized_slice')

.
----------------------------------------------------------------------
Ran 1 test in 0.014s

OK


In [10]:
single_tester(decision_trees_submission_tests.VectorizationWarmUpTests, 'test_vectorized_slice_time')

.
----------------------------------------------------------------------
Ran 1 test in 0.015s

OK


3. `vectorized_flatten()`

In [11]:
single_tester(decision_trees_submission_tests.VectorizationWarmUpTests, 'test_vectorized_flatten')

.
----------------------------------------------------------------------
Ran 1 test in 0.041s

OK


In [13]:
single_tester(decision_trees_submission_tests.VectorizationWarmUpTests, 'test_vectorized_flatten_time')

.
----------------------------------------------------------------------
Ran 1 test in 0.021s

OK


4. `vectorized_glue()`

In [14]:
single_tester(decision_trees_submission_tests.VectorizationWarmUpTests, 'test_vectorized_glue')

.
----------------------------------------------------------------------
Ran 1 test in 0.032s

OK


In [15]:
single_tester(decision_trees_submission_tests.VectorizationWarmUpTests, 'test_vectorized_glue_time')

.
----------------------------------------------------------------------
Ran 1 test in 0.014s

OK


5. `vectorized_mask()`

In [16]:
single_tester(decision_trees_submission_tests.VectorizationWarmUpTests, 'test_vectorized_mask')

.
----------------------------------------------------------------------
Ran 1 test in 0.047s

OK


In [17]:
single_tester(decision_trees_submission_tests.VectorizationWarmUpTests, 'test_vectorized_mask_time')

.
----------------------------------------------------------------------
Ran 1 test in 0.015s

OK


## 任务

分类在机器学习中被广泛用于研究如何对新数据进行分类。

我们将构建、训练和测试决策树模型来执行基本的分类任务。为此，我们需要了解决策树和随机森林是如何工作的。

我们需要一种明确的方法来做出结构化的决策。DecisionNode 类将用于把决策节点表示为多分类决策图中的某个原子级别的选择。

 'DecisionNode' 类型的对象可以表示为一个

  * 决策节点
     - left: 'DecisionNode'类，将指向小于或等于分割值的节点, 即决策表达式判定为真时
     - right: 'DecisionNode'类，将指向大于分割值的节点, 即决策表达式判定为假时
     - decision_function: 计算属性的值并将每个向量映射到下一个节点
     - class_label: None
  * 叶节点
     - left: None
     - right: None
     - decision_function: None
     - class_label: 分类值


### Part 1a:  手动构建决策树

在 `build_decision_tree()` 函数中, 手动构造一个决策树，以便对下面的数据进行分类，即将每个数据**x**映射到一个标签**y**。构建决策树时，应该尽可能使用索引小的属性。

<p>
<img src="./files/Decision_tree_hand.png" alt="Decision Trees" width="500" height="500"/>

#### 要求:
为了使分类更高效，我们可以作出限制：决策树中的元素(节点、叶子)的总数应该小于10。

#### 要点:

我们可以选择创建一个lambda表达式，将其作为 `DecisionNode` 的决策函数，如下所示:

    func = lambda feature : feature[2] <= 0.356

这表明当第三个属性的值小于或等于 0.356 时，将会选择左节点。

举一个例子, 下面是一个决策树:
                                                                 
    func = lambda feature : feature[0] <= -0.918                                                                 

> 在这个例子中，如果 feature[0] 被决策函数评估为 true 那么待分类对象将会被分类到 class = 1，否则 class = 0。
> <p>
> <img src="./files/tree_example.png" alt="Tree Example"/>

我们可以写出下面的代码，来构建上面的决策树:

    func0 = lambda feature : feature[0] <= -0.918
    decision_tree_root = DecisionNode(None, None, func0, None)
    decision_tree_root.left = DecisionNode(None, None, None, class1)
    decision_tree_root.right = DecisionNode(None, None, None, class0)
    return decision_tree_root

#### 需要在 `submission` 文件中完成的函数:
1. `build_decision_tree()`

---

In [18]:
single_tester(decision_trees_submission_tests.DecisionTreePart1Tests, 'test_hand_tree_accuracy')

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK



### Part 1b: 混淆矩阵、准确率、精确率和召回率

混淆矩阵( Confusion Matrix )的大小应该是K × K，其中 K 是类的数量。数据集的真实标签将由行表示，预测标签由列表示。显然，正确的分类预测形成了矩阵的对角线。

准确率( Accuracy )反映检测模型对所有类别的检测性能，其大小为预测正确的次数除以总预测次数。

精确率( Precision) 反映了正确预测的 P(某个类别) 的个数与该模型预测的 P 的样本的个数之比（不管预测的正不正确，只要是 P 就行）。精确率衡量的是模型预测 P 的准确率（相比于准确率，更精确到某一类）。

召回率( Recall )表示为模型正确预测的 P(某个类别) 的个数与真实标签中 P 的样本个数的比值。它用来衡量该模型检测出 P 样本的能力。召回率越高，说明该模型越能检测出 P 样本。

#### 需要在 `submission` 文件中完成的函数:

1. `confusion_matrix()`
2. `accuracy()`
3. `precision()`
4. `recall()`


1. `confusion_matrix()`

In [19]:
single_tester(decision_trees_submission_tests.DecisionTreePart1Tests, 'test_confusion_matrix')

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


2. `precision()`

In [20]:
single_tester(decision_trees_submission_tests.DecisionTreePart1Tests, 'test_precision_calculation')

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


3. `recall()`

In [21]:
single_tester(decision_trees_submission_tests.DecisionTreePart1Tests, 'test_recall_calculation')

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


4. `accuracy()`

In [22]:
single_tester(decision_trees_submission_tests.DecisionTreePart1Tests, 'test_accuracy_calculation')

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK



### Part 2a:  基尼不纯度

通过对数据进行划分，能够使它更加有序。对于有序数据，我们可以获得关于所观察到的事物的属性的更多信息。

在 `gini_impurity()` 函数中，我们将使用基尼不纯度类表征数据的信息熵。

在 `gini_gain()` 函数中，我们将数据进行划分，分别计算每一组数据的基尼不纯度，划分后的数据的信息熵用这些基尼不纯度的加权和来表征，接着，我们计算划分前后的信息熵的增益。

#### 需要在 `submission` 文件中完成的函数:
1. `gini_impurity()`
2. `gini_gain()`


1. `gini_impurity()`

In [23]:
single_tester(decision_trees_submission_tests.DecisionTreePart2Tests, 'test_gini_impurity_max')

.
----------------------------------------------------------------------
Ran 1 test in 0.034s

OK


In [24]:
single_tester(decision_trees_submission_tests.DecisionTreePart2Tests, 'test_gini_impurity_min')

.
----------------------------------------------------------------------
Ran 1 test in 0.019s

OK


In [25]:
single_tester(decision_trees_submission_tests.DecisionTreePart2Tests, 'test_gini_impurity')

.
----------------------------------------------------------------------
Ran 1 test in 0.020s

OK


2. `gini_gain()`

In [26]:
single_tester(decision_trees_submission_tests.DecisionTreePart2Tests, 'test_gini_gain')

.
----------------------------------------------------------------------
Ran 1 test in 0.019s

OK


In [27]:
single_tester(decision_trees_submission_tests.DecisionTreePart2Tests, 'test_gini_gain_max')

.
----------------------------------------------------------------------
Ran 1 test in 0.020s

OK


In [28]:
single_tester(decision_trees_submission_tests.DecisionTreePart2Tests, 'test_gini_gain_restaurant_patrons')

.
----------------------------------------------------------------------
Ran 1 test in 0.020s

OK


In [29]:
single_tester(decision_trees_submission_tests.DecisionTreePart2Tests, 'test_gini_gain_restaurant_type')

.
----------------------------------------------------------------------
Ran 1 test in 0.020s

OK


### Part 2b: 决策树学习

用于训练和测试的数据: **_simple_binary.csv, simple_multi.csv, mod_complex_binary.csv, mod_complex_multi.csv_**

现在，我们需要构建一个决策树类来进行训练，这要求：
   - 初始化决策树的根节点，设定决策树的最大深度。
   - 使用 `fit()` 方法构建决策树。
   - 使用 `__build_tree__()` 方法构建决策树中的节点。
   - 使用 `classify()` 方法进行预测。

构建决策树的一些关键点:

   1. 统计各分类的数量：
      - 如果所有分类相同，可直接确定带有分类标签的叶节点。
      - 如果当前深度超过决策树的深度限制时，选择分类数量最多的分类作为叶节点的标签。
      - 如果不属于以上两种情况，则可能需要在当前节点进行分裂。
   2. 通过某种方式对数据进行划分，并计算信息熵的增益。这里，我们考虑将数据在某个属性的均值作为划分的阈值，当然，数据的划分方式是有限的，我们可以穷尽这些划分，但是采用均值进行划分亦不失为一种好的划分方法。
   3. 我们选择在某个属性进行划分，并将该属性的均值作为阈值，选择它的原因在于，相比其他属性，它具有最高的信息熵增益。
   4. 在信息熵增益过小的情况下，不进行分裂。
   4. 使用 `DecisionNode` 类创建一个决策节点，其决策函数由前面的划分所确定。
   5. 使用递归构建子树，子树的训练集由数据划分得到。

测试说明：训练和待预测的数据完全一致，因此测试要求的准确率为100%。

#### 需要在 `DecisionTree` 类中完成的方法：
1. `__build_tree__()`
2. `fit()`
3. `classify()`


In [30]:
single_tester(decision_trees_submission_tests.DecisionTreePart2Tests, 'test_decision_tree_all_data')

.
----------------------------------------------------------------------
Ran 1 test in 0.210s

OK


### Part 2c:  验证集

使用的数据: **_mod_complex_multi.csv_**

在机器学习建模过程中，通行的做法是将数据分为训练集和测试集。测试集是与训练独立的数据，完全不参与训练，用于最终模型的评估。在训练过程中，经常会出现过拟合的问题，就是模型可以很好的匹配训练数据，却不能很好在预测训练集外的数据。如果此时就使用测试数据来调整模型参数，就相当于在训练时已知部分测试数据的信息，会影响最终评估结果的准确性。通常的做法是在训练数据再中分出一部分做为验证(Validation)数据，用来评估模型的训练效果。

验证数据取自训练数据，但不参与训练，这样可以相对客观的评估模型对于训练集之外数据的匹配程度。模型在验证数据中的评估常用的是交叉验证，又称循环验证。它将原始数据分成K组(K-Fold)，将每个子集数据分别做一次验证集，其余的 K-1 组子集数据作为训练集，这样会得到K个模型。这K个模型分别在验证集中评估结果，最后的误差MSE(Mean Squared Error)相加求平均就得到交叉验证误差。交叉验证有效利用了有限的数据，并且评估结果能够尽可能接近模型在测试集上的表现，可以做为模型优化的指标使用。

在 `generate_k_folds()` 函数中，我们随机地将数据集分成k个大小相同的子部分。每个样本由 k-1 个子部分（训练集）和 1 个子部分（验证集）构成，显然，我们将得到 k 个互不相同的样本。

然后对每个样本进行迭代，使用其训练集进行训练，保留验证集用于测试。对测试结果取平均值。

#### 需要在 `submission` 文件中完成的函数:
1. `generate_k_folds()`


In [31]:
single_tester(decision_trees_submission_tests.DecisionTreePart2Tests, 'test_k_folds_training_set_count')

.
----------------------------------------------------------------------
Ran 1 test in 0.024s

OK


In [32]:
single_tester(decision_trees_submission_tests.DecisionTreePart2Tests, 'test_k_folds_test_set_count')

.
----------------------------------------------------------------------
Ran 1 test in 0.024s

OK


### Part 3:  随机森林

使用的数据: **_mod_complex_binary.csv, mod_complex_multi.csv_**

期望: k = 10，kFold-交叉验证的平均准确率 >= 80%

决策树描述的决策边界非常清晰，将深度无限的决策树拟合到训练集几乎不可避免地会导致过拟合。为了避免这种问题，我们将使用一种称为 "Bootstrap Aggregating" (通常缩写为 "bagging" ) 的技术。

随机森林属于 bagging 方法，它是决策树的集合（不同决策树之间没有关联），它的构建步骤如下:

1. 一个样本容量为 M 的样本，有放回的抽取 m 次，每次抽取1个，最终形成了 m 个样本。这选择好了的 m 个样本用来训练一个决策树，作为决策树根节点处的样本。
2. 当每个样本有 N 个属性时，在决策树的每个节点需要分裂时，随机从这 N 个属性中选取出 n 个属性，满足条件n << N。然后从这 n 个属性中采用某种策略（比如说信息增益）来选择1个属性作为该节点的分裂属性。
3. 决策树形成过程中每个节点都要按照步骤 2 来分裂（很容易理解，如果下一次该节点选出来的那一个属性是刚刚其父节点分裂时用过的属性，则该节点已经达到了叶子节点，无须继续分裂了）。一直到不能够再分裂为止。注意整个决策树形成过程中没有进行剪枝。
4. 按照步骤 1~3 建立大量的决策树，这样就构成了随机森林了。

现在，我们需要构建一个随机森林类来进行训练，这要求：
   - 初始化随机森林包含的决策树总数，设定决策树的最大深度，设定样本抽样率(m/M)和属性抽样率(n/N)。
   - 使用 `fit()` 方法构建随机森林中的所有决策树。
   - 使用 `classify()` 方法进行预测，收集每个决策树的预测，并将出现最多的预测类作为随机森林的预测结果。

测试时, 我们将设定决策树总数为 80，最大深度为 5，样本抽样率为 0.3，属性抽样率为 0.3。

#### 需要在 `RandomForest` 类中完成的方法:
1. `fit()`
2. `classify()`

In [33]:
single_tester(decision_trees_submission_tests.DecisionTreePart3Tests, 'test_binary_random_forest')

.

0.9700000000000001



----------------------------------------------------------------------
Ran 1 test in 3.698s

OK


In [34]:
single_tester(decision_trees_submission_tests.DecisionTreePart3Tests, 'test_multi_random_forest')

.

0.8908333333333331



----------------------------------------------------------------------
Ran 1 test in 10.295s

OK


### Part 4: 挑战（待完成）

使用的数据: **_complex_binary.csv, complex_multi.csv_**

期望: k = 10，kFold-交叉验证的平均准确率 >= 80%
   
决策树描述的决策边界非常清晰，将深度无限的决策树拟合到训练集几乎不可避免地会导致过拟合。为了避免这种问题，可以使用一种名为 'Boosting' 的技术。 

挑战性在于，我们尝试实现一种 boosting 算法，比如 Ada-, Gradient-, XG-, boost。与随机森林相类似的是，这些算法将使用深度较浅的决策树的集合。

这些算法的一些特点：
    - 它们通常很短(深度有限)
    - 它们使用随机数据集的抽样进行训练
    - 它们使用经过抽样的属性子集
    - 它们能够很好地拟合经过抽样的数据集
    - 它们使用多数投票(森林中的每棵树都投票)来分类样本

完成 `Challeclassifier.fit()` 来拟合我们上面描述的决策树，并填写 `Challeclassifier.classifier()` 来对示例进行分类。

测试时, 我们将设定决策树总数为 200，最大深度为 3，样本抽样率为 0.1，属性抽样率为 0.1。

#### 可能需要在 `ChallengeClassifier` 类中完成的方法:

1. `fit()`
2. `boost()`
3. `classify()`


In [35]:
single_tester(decision_trees_submission_tests.DecisionTreePart4Tests, 'test_binary_boosting')

.

0.865



----------------------------------------------------------------------
Ran 1 test in 4.179s

OK


In [36]:
single_tester(decision_trees_submission_tests.DecisionTreePart4Tests, 'test_multi_boosting')

F

0.29624999999999996



FAIL: test_multi_boosting (decision_trees_submission_tests.DecisionTreePart4Tests)
Test random forest on binary data.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\san\Desktop\Decision-Trees\decision_trees_submission_tests.py", line 422, in test_multi_boosting
    assert sum(results) / 10. >= .80
AssertionError

----------------------------------------------------------------------
Ran 1 test in 6.432s

FAILED (failures=1)
